In [1]:
#mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls

Mounted at /content/drive
drive  sample_data


In [2]:
# move into project directory
repo_name = "Image-Colorization"
%cd /content/drive/MyDrive/Personal-Projects/$repo_name
!ls

/content/drive/MyDrive/Personal-Projects/Image-Colorization
common	     dataloading  Index.ipynb  output		     README.md	       transforms
config.yaml  experiments  index.py     preprocess	     requirements.txt  visualization
data	     Index_bc.py  models       project-structure.md  run.yaml


In [3]:
# set up environment
# comment out if not required
'''
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install matplotlib numpy pandas pyyaml opencv-python
'''

'\n!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118\n!pip install matplotlib numpy pandas pyyaml opencv-python\n'

In [4]:
# this cell is for downloading data.
# as of yet data is not hosted and is available in the private data folder


In [5]:
# setup some imports
#custom imports
from transforms.transforms import ToTensor
from dataloading.datareader import DataReader
from dataloading.dataset import CustomDataset
from common.utils import get_exp_params, init_config, get_config, save2config
from models.unet import UNet

#py imports
import random
import numpy as np
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from experiments.experiments import Experiment
from visualization.visualization import Visualization

In [6]:
# read experiment parameters
exp_params = get_exp_params()
print('Experiment parameters\n')
print(exp_params)

Experiment parameters

{'transform': {'resize_dim': 256, 'crop_dim': 224}, 'train': {'shuffle_data': True, 'batch_size': 128, 'val_split_method': 'k-fold', 'k': 5, 'val_percentage': 20, 'loss': 'mse', 'batch_interval': 512, 'epoch_interval': 1, 'num_epochs': 20}, 'model': {'name': 'unet', 'optimizer': 'Adam', 'lr': 0.1, 'weight_decay': 1e-08, 'amsgrad': False, 'momentum': 0.9}, 'test_model': False}


In [7]:
# initialize directories and config data
init_config()
config = get_config()
print('Config parameters\n')
print(config)

Config parameters

{'X_key': 'L', 'data_dir': '/content/drive/MyDrive/Personal-Projects/Image-Colorization/data', 'root_dir': '/content/drive/MyDrive/Personal-Projects/Image-Colorization', 'use_gpu': True, 'y_key': 'AB'}


In [8]:
#initialize randomness seed
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [9]:
#save X_key and y_key
save2config('X_key', 'L')
save2config('y_key', 'AB')

#preprocess data or load preprocessed data
dr = DataReader()
ds = dr.get_split_data()
Ltr, ABtr, ftr_len = ds['Ltr'], ds['ABtr'], ds['ftr_len']
Lte, ABte, te_len = ds['Lte'], ds['ABte'], ds['te_len']
print(ds['Ltr'].shape, ds['ABtr'].shape)
print('Full Train data length:', ftr_len)
print('Test data length:', te_len, '\n')

#transform data
composed_transforms =  transforms.Compose([
    ToTensor(True)
])

#convert to dataset
ftr_dataset = CustomDataset(Ltr, ABtr, ftr_len, composed_transforms)
te_dataset = CustomDataset(Lte, ABte, te_len, composed_transforms)


split data (50000, 32, 32, 2) (50000, 32, 32)
(50000, 1, 32, 32) (50000, 2, 32, 32)
Full Train data length: 50000
Test data length: 10 



In [ ]:
#model training

if exp_params['model']['name'] == 'unet':
    model = UNet()
else:
    raise SystemExit("Error: Invalid model name passed! Check run.yaml")

exp = Experiment(model, ftr_dataset)
model_info = exp.train()
print("\nModel validation results")

#visualization results
vis = Visualization(model_info)
vis.get_results()


Running split 0
	Running Epoch 0
		Running through train dataset
		Running through validation dataset
	Epoch 0 Training Loss: 0.020576046761898947
	Epoch 0 Validation Loss: 0.021082169526622077
	Epoch 0 Validation Accuracy: 0.0

	Running Epoch 1
		Running through train dataset
		Running through validation dataset
	Epoch 1 Training Loss: 0.019864943605987503
	Epoch 1 Validation Loss: 0.020634228982550056
	Epoch 1 Validation Accuracy: 0.0

	Running Epoch 2
		Running through train dataset
		Running through validation dataset
	Epoch 2 Training Loss: 0.019808569081728688
	Epoch 2 Validation Loss: 0.020141038018031212
	Epoch 2 Validation Accuracy: 0.0

	Running Epoch 3
		Running through train dataset
		Running through validation dataset
	Epoch 3 Training Loss: 0.01968140251705417
	Epoch 3 Validation Loss: 0.02010065930176385
	Epoch 3 Validation Accuracy: 0.0

	Running Epoch 4
		Running through train dataset
		Running through validation dataset
	Epoch 4 Training Loss: 0.019629321825027038
	Ep